# Entwicklung eines Prototyps zur Münzzählung mithilfe von Bildverarbeitung und Machine Learning
## Development of a prototype for coin counting using image processing and machine learning

author:  Angmar3019 <br>
date:    07.02.2023 <br>
version: 1.0.0 <br>
licence: GNU General Public License v3.0 <br>

## Clone repository

In [ ]:
!git clone https://github.com/Angmar3019/DHBW-Studienarbeit.git

## Upload images and coco dataset

Either you can take the provided images from me, then execute the first cell.

If you want to use your own images, then execute the second cell and then upload your images to the folder `/data/images/` and save your coco dataset under `/data/result.json`.

Then you can execute the other cells.

In [ ]:
# Using providet images
!mkdir -p data/images
!cp -r /content/DHBW-Studienarbeit/training/* /content/data/

In [ ]:
# Using own images
!mkdir -p data/images
!cp /content/DHBW-Studienarbeit/training/label_map.txt /content/data/
#Upload your images and coco dataset

## Installing requirements

In [ ]:
#Restart of google collab session required
!pip install tensorflow==2.15.0

In [ ]:
! rm -rf sample_data

In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler

In [ ]:
!git clone https://github.com/tensorflow/models.git

%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd /content

In [ ]:
!pip install tflite
!pip install tflite_runtime

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

!tar -xvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!rm ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

model = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"

In [ ]:
!mkdir model_tfrecord
!mkdir model_trained
!mkdir model_exported

## Model training

In [ ]:
import os
import json
import random

import tensorflow as tf
import numpy as np

from PIL import Image

In [ ]:
def adjust_file_names(input_file):
    """Adjust file names in COCO dataset
    - !!!Only needed if you use Label Studio!!!
    - Removes the prefix "image/" from the names of the images
    - Adds the category "background" with id=0
    - Moves all other categories up by one id
    - Exports customized COCO dataset xml

    Args:
      - input_file (str):   Original COCO dataset from Label Studio

    Test:
      - Check if background category has been added
      - Check if the prefix is missing in the filename
    """

    output_file="/content/data/labels.json"
    with open(input_file, 'r') as f:
        data = json.load(f)

    data['categories'] = [{'id': 0, 'name': 'background'}] + data['categories']

    for i, category in enumerate(data['categories'][1:], start=1):
        category['id'] = i

    for annotation in data['annotations']:
        if annotation['category_id'] > 0:
            annotation['category_id'] += 1

    for image in data['images']:
        image['file_name'] = os.path.basename(image['file_name'])

    with open(output_file, 'w') as f:
        json.dump(data, f)

    print(f"Die Datei {input_file} wurde erfolgreich angepasst und gespeichert als {output_file}")



adjust_file_names("/content/data/result.json")

In [ ]:
def split_coco_dataset(image_dir, result_json, train_percent, val_percent, test_percent):
    """Split COCO dataset
    - Splits a COCO dataset into train, val, and test sets
    - Adjust the input percentages according to your splitting needs

    Args:
      - image_dir (str):      Directory containing the images.
      - result_json (str):    Path to the COCO JSON file.
      - train_percent (int):  Percentage of images used for training
      - val_percent (int):    Percentage of images used for validation
      - test_percent (int):   Percentage of images used for testing

    Test:
      - Check whether the three different json have been created in "/content/data"
    """

    with open(result_json, 'r') as file:
        data = json.load(file)

    images = data['images']
    annotations = data['annotations']

    random.shuffle(images)
    total_images = len(images)
    train_end = int(total_images * train_percent / 100)
    val_end = train_end + int(total_images * val_percent / 100)

    train_images = images[:train_end]
    val_images = images[train_end:val_end]
    test_images = images[val_end:]



    def filter_annotations(selected_images):
        image_ids = set([img['id'] for img in selected_images])
        return [ann for ann in annotations if ann['image_id'] in image_ids]



    def save_coco_file(images, annotations, filename):
        new_data = {
            'images': images,
            'annotations': annotations,
            'categories': data['categories']
        }
        with open(filename, 'w') as file:
            json.dump(new_data, file)

        print(f"Exported splitted COCO dataset to {filename}")

    train_annotations = filter_annotations(train_images)
    val_annotations = filter_annotations(val_images)
    test_annotations = filter_annotations(test_images)

    save_coco_file(train_images, train_annotations, "/content/data/train.json")
    save_coco_file(val_images, val_annotations, "/content/data/val.json")
    save_coco_file(test_images, test_annotations, "/content/data/test.json")



split_coco_dataset('/content/data/images', '/content/data/labels.json', 70, 20, 10)

In [ ]:
!rm -f /content/tfrecord/*
%cd models
!python research/object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
  --train_image_dir="/content/data/images" \
  --val_image_dir="/content/data/images" \
  --test_image_dir="/content/data/images" \
  --train_annotations_file="/content/data/train.json" \
  --val_annotations_file="/content/data/val.json" \
  --testdev_annotations_file="/content/data/test.json" \
  --output_dir="/content/model_tfrecord/" \
%cd /content/

In [ ]:
def update_config_file(config_path, fine_tune_checkpoint, train_record_path, val_record_path, label_map_path, batch_size, steps, num_classes):
    """Update config file
    - !!!If you change the model, the replacement functions must be adapted accordingly!!!
    - Adjusts the values and paths from the original model config and outputs a new "pipeline.config"

    Args:
      - config_path (str):          Path to the .config file to be updated
      - fine_tune_checkpoint (str): Path to the pre-trained model checkpoint
      - train_record_path (str):    Path to the training TFRecord file
      - val_record_path (str):      Path to the validation TFRecord file
      - label_map_path (str):       Path to the label map file
      - batch_size (int):           Number of batch count
      - steps (int):                Number of streps
      - num_classes (int):          Number of classes

    Test:
      - Check whether the paths and values in the output "pipeline.config" are correct
    """

    with open(config_path) as f:
        config = f.read()

    config = config.replace('fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED"', f'fine_tune_checkpoint: "{fine_tune_checkpoint}"')
    config = config.replace('input_path: "PATH_TO_BE_CONFIGURED"', f'input_path: "{train_record_path}"')
    config = config.replace('input_path: "PATH_TO_BE_CONFIGURED/val2017-?????-of-00032.tfrecord"', f'input_path: "{val_record_path}"')
    config = config.replace('label_map_path: "PATH_TO_BE_CONFIGURED"', f'label_map_path: "{label_map_path}"')
    config = config.replace('batch_size: 128', f'batch_size: {batch_size}')
    config = config.replace('total_steps: 50000', f'total_steps: {steps}')
    config = config.replace('num_steps: 50000', f'num_steps: {steps}')
    config = config.replace('num_classes: 90', f'num_classes: {num_classes}')
    config = config.replace('fine_tune_checkpoint_type: "classification"', f'fine_tune_checkpoint_type: "detection"')

    with open("/content/pipeline.config", 'w') as f:
        f.write(config)

    print(f"{config_path} was updated with the specified parameters")



update_config_file(
    config_path='/content/' + model + '/pipeline.config',
    fine_tune_checkpoint='/content/' + model + '/checkpoint/ckpt-0',
    train_record_path='/content/model_tfrecord/coco_train*',
    val_record_path='/content/model_tfrecord/coco_val*',
    label_map_path='/content/data/label_map.txt',
    batch_size=4,
    steps=5000,
    num_classes=9
)

In [ ]:
# Source: https://github.com/tensorflow/models/issues/11099#issuecomment-1902615454

import shutil
import re

original_path = '/usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py'
with open(original_path, 'r') as file:
  content = file.read()
  content = re.sub(r'import abc', 'import tensorflow as tf\n\nimport abc', content)
  content = re.sub(r'control_flow_ops.case', 'tf.case', content)
  content = re.sub(r'control_flow_ops.cond', 'tf.compat.v1.cond', content)
with open(original_path, 'w') as file:
  file.write(content)

print(f"File {original_path} fixed.")

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/trained

In [ ]:
%cd /content/models/research/object_detection
!python model_main_tf2.py --model_dir /content/model_trained --pipeline_config_path /content/pipeline.config
%cd /content/

In [ ]:
%cd models/research/object_detection
!python export_tflite_graph_tf2.py --output_directory /content/model_exported --pipeline_config_path /content/pipeline.config --trained_checkpoint_dir /content/model_trained/
%cd /content/

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("/content/model_exported/saved_model")
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]

tflite_model = converter.convert()

with open('/content/model_tensorflow.tflite', 'wb') as f:
    f.write(tflite_model)

print("Exported and converted Tensorflow model into model_tensorflow.tflite")

## Post-training quantization

In [ ]:
def representative_data_gen():
    """Loads representative dataset
    - Loads the images from the COCO dataset train.json for the quantization process

    Test:
      - Check whether the conversion process was successful
    """

    annotation_file = '/content/data/test.json'
    image_dir = '/content/data/images'

    with open(annotation_file, 'r') as f:
        annotations_data = json.load(f)

    for image_info in annotations_data['images']:
        image_path = os.path.join(image_dir, image_info['file_name'])
        image = np.array(Image.open(image_path))
        image = tf.image.resize(image, [320, 320])
        image = tf.cast(image / 255., tf.float32)
        image = tf.expand_dims(image, 0)
        yield [image]



converter = tf.lite.TFLiteConverter.from_saved_model("/content/model_exported/saved_model")

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_types = [tf.int8]

converter.inference_input_type = tf.int8
converter.inference_output_type = tf.float32

tflite_model = converter.convert()

with open('/content/model_quant.tflite', 'wb') as f:
  f.write(tflite_model)

print("Exported and quantized model into model_quant.tflite")

## Conversion to an EdgeTPU model

In [ ]:
!edgetpu_compiler /content/model_quant.tflite